<a href="https://colab.research.google.com/github/jenajjedu/NLP_Named_Entities/blob/main/PA6_part_2_studentGhalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART 2: LSTM FOR CLASSIFICATION**

**Task 1:**

Follow the tutorial [here](https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb) on how to build an LSTM model for sentiment classification. Modify the tutorial to train on
your tweet sentiment data (sentiment-train.csv) and test on test data (sentiment-test.csv) from HW2 (modify the tutorial
so that the training data is not split into training and validation). Compute and report the accuracy on the test data.

**Checkpoint 1:**

By following block 1-14, you should be able to build the inputs and labels of the sentiment data. The processed inputs and labels should look similar to the following.

In [ ]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
file = open("sentiment-test.csv", "r")
#read in as pd data frame

#train_df = sentiment-test.csv

#train`_label = train_df['sentiment].strlower().tolist()

data = list(csv.reader(file, delimiter=","))
file.close()

print(data)

[['sentiment', 'text'], ['1', '@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'], ['1', 'Reading my kindle2...  Love it... Lee childs is good read.'], ['1', 'Ok, first assesment of the #kindle2 ...it fucking rocks!!!'], ['1', "@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)"], ['1', "@mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)"], ['1', "@richardebaker no. it is too big. I'm quite happy with the Kindle2."], ['1', 'Jquery is my new best friend.'], ['1', 'Loves twitter'], ['1', 'how can you not love Obama? he makes jokes about himself.'], ['1', 'House Correspondents dinner was last night whoopi, barbara &amp; sherri went, Obama got a standing ovation'], ['1', 'Watchin Espn..Jus seen this new Nike Commerical with a Puppet Lebron..sh*t was hilarious...LMAO!!!'], ['1', "#lebron best athlete of our generati

In [ ]:
file2 = open("sentiment-train.csv", "r")
data2 = list(csv.reader(file2, delimiter=","))
file2.close()

print(data2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import csv
#import fishing
def loadTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['sentiment'])] = row['text']
  return tweetDict

#load your fishing tweet data here:
csvPathtest = "sentiment-test.csv"
rawTweetDicttest = loadTextFromCSV(csvPathtest)


#print your tweet dictionary. You should see your saved tweets inside.
print("rawTweetDictFish: ",rawTweetDicttest)


rawTweetDictFish:  {1: 'Ahhh... back in a *real* text editing environment. I &lt;3 LaTeX.', 0: 'Reading the tweets coming out of Iran... The whole thing is terrifying and incredibly sad...'}


In [ ]:


print("Number of training reivews: " + str(len(data)))
print("Number of test reviews: " + str(len(data2)))



Number of training reivews: 360
Number of test reviews: 60001


In [ ]:
num_train = 800000 #We're training on the first 800,000 reviews in the dataset
num_test = 200000 #Using 200,000 reviews from test set

print(data[0])

['sentiment', 'text']


In [ ]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in data]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in data]

    
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in data2]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in data2]

# Some simple cleaning of data

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

# Modify URLs to 

for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "", test_sentences[i])

AttributeError: ignored

In [ ]:
words = Counter() #Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    #The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence): #Tokenizing the words
        words.update([word.lower()]) #Converting all the words to lower case
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done")
print("100% done")


In [ ]:


# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}



In [ ]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else word2idx['_UNK'] for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else word2idx['_UNK'] for word in nltk.word_tokenize(sentence)

In [ ]:
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [ ]:
seq_len = 200 #The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

train_sentences[:2]: 
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0   

**Checkpoint 2:**

By following block 16-18, you should be able to build the dataset and dataloader from Pytorch which loads your inputs and labels and turns them into batches. A batch of the inputs/labels should look like the following.

In [ ]:


import torch
from torch.utils.data import TensorDataset, DataLoader



In [ ]:
train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")


GPU not available, CPU used


In [ ]:


dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print(sample_x.shape, sample_y.shape)



GPU not available, CPU used
torch.Size([400, 200]) torch.Size([400])


**Checkpoint 3:**

By following block 19 and 20, you should be able to set up the LSTM model for classification, the structure should look like the following.

In [ ]:


import torch.nn as nn

class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden



In [ ]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)
print(model)

SentimentNet(
  (embedding): Embedding(19554, 400)
  (lstm): LSTM(400, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


**Checkpoint 4:**

By following block 21-22, train your model.

In [ ]:
lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

NameError: ignored

In [ ]:


epochs = 2
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)



**Checkpoint 5:**

By following block 25, you should be able to implement the evaluation. The evaluation result should look like the following.

In [ ]:


#Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze()) #rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
        
print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))



Test loss: 0.507
Test accuracy: 74.652%


**Task 2:**

Modify the model from task 1 to use GRU instead of LSTM. Compute and report the accuracy on the test data.

NOTE: at this point, you should be able to finish the task without checkpoints. Try to modify/re-use the function you implemented in task 1.

**Task 3:**

Modify the model from task 1 to use bidirectional LSTM. Compute and report the accuracy on the test data.

NOTE: Try modify/re-use the function you implemented in task 1, 2.

**Task 4:**

Modify the model from 3.1 to use bidirectional GRU. Compute and report the accuracy on the test data.

**Task 5:**

Pick the best model so far and train the model starting from pretrained [GloveTwitter 100d](https://nlp.stanford.edu/projects/glove/) (from 'glove.twitter.27B', use the same vocabulary
as before, just initialize the embedding of the words using Glove embeddings). Compute and report the accuracy on the
test data.

NOTE: the modification on the model should be very similar to the one you did in part one for LSTM language modeling with glove 100d.

**Task 6:**

Using your best model so far, conduct a 5-fold (stratified) cross validation on your training data and a grid
search to pick the best hidden size (try 128 or 512) and embedding size (try 100 or 400). Compute and report the average
accuracies for each of the choice combination. 

NOTE: You don't need glove for word embeddings in this task.

HINT: you can implement this by using the 'StratifiedKFold' from sklearn

With hyperparam 512, 400:

With hyperparam 512, 100:

With hyperparam 128, 400:

With hyperparam 128, 100:

**Task 7:**

Train the model on all your training data using the best combination of hyperparameters you find in task 6. Compute
and report the accuracy on the test data.